****Step 1: Imports****

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np


**Step 2: Identity Block (No Conv Shortcut)**

In [10]:
def identity_block(X,f,filters,training=True):
  #Split the list of filters into 3 variables — each will be used in one Conv layer in the main path.
  F1,F2,F3=filters

  #Save a copy of the input X. This will be added back later (residual connection)
  X_shortcut=X

  #Main Path

  #Applies a 1×1 convolution to reduce or change the number of channels.
  #No padding (valid), so output shape may be smaller.
  #Number of output channels = F1.
  X = Conv2D(filters = F1,kernel_size=(1,1),padding='valid')(X)

  #Normalizes the output to speed up training and reduce internal covariate shift.
  X = BatchNormalization(axis=3)(X, training=training)

  #Applies ReLU non-linearity to add activation and remove negative values.
  X = Activation('relu')(X)

  #2nd Layer: f x f Convolution (middle layer)

  #Applies a f × f convolution (usually f=3).
  #Keeps spatial size the same (padding='same').
  #Output channels = F2.
  X = Conv2D(filters = F2,kernel_size=(f,f),strides=(1,1),padding='same')(X)

  #Normalize again followed by ReLU
  X = BatchNormalization(axis=3)(X,training=training)
  X = Activation('relu')(X)

  #3rd Layer: 1x1 Convolution (restore channels)
  X = Conv2D(filters = F3,kernel_size=(1,1),strides=(1,1),padding='valid')(X)
  X = BatchNormalization(axis=3)(X,training=training)

  #Add Shortcut
  #Adds the original input (X_shortcut) to the output of the main path
  #This is the "skip connection" or residual connection.

  X = Add()([X,X_shortcut])

  #Applies ReLU after adding the shortcut. This is the final output of the identity block.
  X = Activation('relu')(X)

  return X

**Step 3: Convolutional Block (With Conv Shortcut)**

In [11]:
def convolutional_block(X,f,filters,s=2,training=True):
  """
  X             Input tensor (e.g., a feature map)
  f             Size of the middle convolution kernel (usually 3)
  filters         List of 3 integers: number of filters for each layer
  s             Stride used in the first Conv layer and the shortcut path (typically 2 for downsampling)
  training      Whether the model is in training mode (used for BatchNorm)

  """
  #Unpack filter sizes
  #Assigns the filter sizes for each convolution layer in the main path.
  F1,F2,F3 = filters

  # Save the shortcut
  # Stores the original input so it can be added later (residual connection).
  X_shortcut = X

  #Main Path (3 layers)

  #Layer 1: 1×1 Conv (compression + optional downsampling)
  #Reduces channels to F1 and downsamples (if s=2).
  #No padding; output may shrink.
  X = Conv2D(filters = F1,kernel_size=(1,1),strides=(s,s),padding='valid')(X)
  #BatchNorm helps normalize activations.
  X = BatchNormalization(axis=3)(X,training=training)
  #ReLU for non-linearity.
  X = Activation('relu')(X)

  #Layer 2: f×f Conv (spatial filter)
  #Keeps the spatial size (padding='same').
  #Applies a deeper feature extraction with F2 filters of size f×f.
  X = Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same')(X)
  X = BatchNormalization(axis=3)(X,training=training)
  X = Activation('relu')(X)

  #Layer 3: 1x1 Conv (restore depth)
  X = Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid')(X)
  X = BatchNormalization(axis=3)(X,training=training)

  #Shortcut Path (1 layer)
  #Applies a 1×1 convolution to the shortcut connection.
  #This is only done in convolutional blocks to match the dimensions of the main path output.
  X_shortcut = Conv2D(filters = F3,kernel_size=(1,1),strides=(s,s),padding='valid')(X_shortcut)
  X_shortcut = BatchNormalization(axis=3)(X_shortcut,training=training)

  #Add Shortcut
  #Combines the output of the main path and the shortcut path.
  X = Add()([X,X_shortcut])

  #Final Activation
  #Applies ReLU to the combined output.
  X = Activation('relu')(X)

  return X

**Step 4: Build a Tiny ResNet Model (For CIFAR-10 like images)**

In [12]:
from tensorflow.keras.initializers import glorot_uniform

def ResNet50(input_shape=(64,64,3),classes=6):
  """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
  """
  # Define the input as a tensor with shape input_shape
  X_input = Input(input_shape)

  # Zero-Padding
  X = ZeroPadding2D(padding=(3,3))(X_input)

  # Stage 1
  X = Conv2D(filters=64,kernel_size=(7,7),strides=(2,2),kernel_initializer=glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D(pool_size=(3,3),strides=(2,2))(X)

  # Stage 2
  X = convolutional_block(X,f=3,filters=[64,64,256],s=1)
  X = identity_block(X,3,[64,64,256])
  X = identity_block(X,3,[64,64,256])

  # Stage 3
  X = convolutional_block(X,f=3,filters=[256,256,1024],s=2)
  X = identity_block(X, 3, [256, 256, 1024])
  X = identity_block(X, 3, [256, 256, 1024])
  X = identity_block(X, 3, [256, 256, 1024])
  X = identity_block(X, 3, [256, 256, 1024])
  X = identity_block(X, 3, [256, 256, 1024])

  # Stage 4
  X = convolutional_block(X,f=3,filters=[512,512,2048],s=2)
  X = identity_block(X, 3, [512, 512, 2048])
  X = identity_block(X, 3, [512, 512, 2048])

  #AVGPOOL
  X = AveragePooling2D(pool_size=(2,2))(X)

  # Output layer
  X = Flatten()(X)
  X = Dense(classes,activation='softmax',kernel_initializer=glorot_uniform(seed=0))(X)

  #Create model
  model = Model(inputs=X_input,outputs=X)

  return model

In [13]:
model = ResNet50(input_shape=(64,64,3),classes=6)
print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 70, 70, 3) │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │      9,472 │ zero_padding2d_1… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 15, 15,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 15, 15,    │      4,160 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 15, 15,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 15, 15,    │     36,928 │ activation_4[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        256 │ conv2d_7[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 15, 15,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 15, 15,    │     16,640 │ activation_5[0][… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 15, 15,    │     16,640 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │      1,024 │ conv2d_8[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │      1,024 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 15, 15,    │          0 │ batch_normalizat

 Total params: 22,078,854 (84.22 MB)

 Trainable params: 22,032,902 (84.05 MB)

 Non-trainable params: 45,952 (179.50 KB)

None


In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [18]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 160s 5s/step - accuracy: 0.4773 - loss: 1.6189
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 202s 5s/step - accuracy: 0.7204 - loss: 0.7302
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 200s 5s/step - accuracy: 0.8608 - loss: 0.4166
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 160s 5s/step - accuracy: 0.7578 - loss: 2.6891
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 199s 5s/step - accuracy: 0.7672 - loss: 1.1526
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 157s 5s/step - accuracy: 0.8454 - loss: 0.9823
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 204s 5s/step - accuracy: 0.8983 - loss: 0.2691
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 201s 5s/step - accuracy: 0.9388 - loss: 0.1804
Epoch 9/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 159s 5s/step - accuracy: 0.9730 - loss: 0.0803
Epoch 10/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 161s 5s/step - accuracy: 0.9564 - loss: 0.1426


Saved artifact at 'hand_signs.h5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor_14')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  138412649302160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138412649303504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138412649304080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138412649302544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138412649303888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138412649301968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138412649305616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138412649306192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138412649306384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138412649305232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138412649302928:

In [21]:
model.save('hand_signs_model.h5')